In [240]:
import pandas as pd
import numpy as np
import operator
import ast
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch
import torch.nn as nn
import string
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 
import spacy
import re

In [241]:
data = pd.read_csv('combined-newsqa-data-v1.csv')
print(data.columns)
print(type(data['is_question_bad'][0]))

# remove Q/A pairs that are invalid or missing
#data = data[(data.is_question_bad=='0.0') & (data.is_answer_absent=='0.0')]
data.head()

Index(['story_id', 'question', 'answer_char_ranges', 'is_answer_absent',
       'is_question_bad', 'validated_answers', 'story_text'],
      dtype='object')
<class 'str'>


,story_id,question,answer_char_ranges,is_answer_absent,is_question_bad,validated_answers,story_text
0,./cnn/stories/42d01e187213e86f5fe617fe32e716ff...,What was the amount of children murdered?,294:297|None|None,0.0,0.0,"{""none"": 1, ""294:297"": 2}","NEW DELHI, India (CNN) -- A high court in nort..."
1,./cnn/stories/c48228a52f26aca65c31fad273e66164...,Where was one employee killed?,34:60|1610:1618|34:60,0.0,0.0,NaN,(CNN) -- Fighting in the volatile Sudanese reg...
2,./cnn/stories/c65ed85800e4535f4bbbfa2c34d7d963...,who did say South Africa did not issue a visa ...,103:127|114:127|839:853,0.0,0.0,"{""839:853"": 1, ""103:127"": 2}",Johannesburg (CNN) -- Miffed by a visa delay t...
3,./cnn/stories/0cf66b646e9b32076513c050edf32a79...,How many years old was the businessman?,538:550|538:550,0.0,0.0,NaN,(CNN) -- England international footballer Ste...
4,./cnn/stories/13012604e3203c18df09289dfedd14cd...,What frightened the families?,690:742|688:791|630:646,0.0,0.0,"{""688:791"": 2, ""690:742"": 1}","BAGHDAD, Iraq (CNN) -- At least 6,000 Christi..."


In [242]:
print(data['is_question_bad'][0])

0.0


In [243]:
# 119,633 Q/A's , 12088 articles
print(len(data))
print(len(data['story_text'].unique()))


119633
12088


In [244]:
first_doc = data['story_text'][0]
first_doc

'NEW DELHI, India (CNN) -- A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed "the house of horrors."\n\n\n\nMoninder Singh Pandher was sentenced to death by a lower court in February.\n\n\n\nThe teen was one of 19 victims -- children and young women -- in one of the most gruesome serial killings in India in recent years.\n\n\n\nThe Allahabad high court has acquitted Moninder Singh Pandher, his lawyer Sikandar B. Kochar told CNN.\n\n\n\nPandher and his domestic employee Surinder Koli were sentenced to death in February by a lower court for the rape and murder of the 14-year-old.\n\n\n\nThe high court upheld Koli\'s death sentence, Kochar said.\n\n\n\nThe two were arrested two years ago after body parts packed in plastic bags were found near their home in Noida, a New Delhi suburb. Their home was later dubbed a "house of horrors" by the Indian media.\n\n\n\nPandher was not named a main suspect by 

In [245]:
first_doc[294:297]

'19 '

In [246]:
second_doc = data['story_text'][2]
second_doc

'Johannesburg (CNN) -- Miffed by a visa delay that led the Dalai Lama to cancel a trip to South Africa, Archbishop Desmond Tutu lashed out at his government Tuesday, saying it had acted worse than apartheid regimes and had forgotten all that the nation stood for.\n\n\n\n\n\n"When we used to apply for passports under the apartheid government, we never knew until the last moment what their decision was," Tutu said at a news conference. "Our government is worse than the apartheid government because at least you were expecting it from the apartheid government.\n\n\n\n\n\n"I have to say that I can\'t believe this. I really can\'t believe this," Tutu said. "You have to wake me up and tell me this is actually happening here."\n\n\n\n\n\nThe Dalai Lama scrapped his planned trip to South Africa this week after the nation failed to issue him a visa in time, his spokesman said.\n\n\n\n\n\nVisa applications for him and his entourage were submitted to the South African High Commission in New Delhi,

In [247]:
print(second_doc[103:127])
print(second_doc[114:127])
print(second_doc[839:853])

Archbishop Desmond Tutu 
Desmond Tutu 
his spokesman 


Answers given by different human reviewers are in the answer_char_ranges column and the validated_answers column. These values are string index ranges within the document that represent the answer. For each row, we need to determine which character range is the best answer to use and extract it. There are a couple ways we could approach this problem, the first is to have a model that takes the question(str), the story(str), spits out an answer(str).

or we could have it spit back out the string indexes themselves. 

In [248]:
# Remove uneeded columns
df = data[['question','validated_answers','story_text']]
df = df.dropna()
df = df.iloc[0:100] # reduce size for development purposes

df.head()

,question,validated_answers,story_text
0,What was the amount of children murdered?,"{""none"": 1, ""294:297"": 2}","NEW DELHI, India (CNN) -- A high court in nort..."
2,who did say South Africa did not issue a visa ...,"{""839:853"": 1, ""103:127"": 2}",Johannesburg (CNN) -- Miffed by a visa delay t...
4,What frightened the families?,"{""688:791"": 2, ""690:742"": 1}","BAGHDAD, Iraq (CNN) -- At least 6,000 Christi..."
6,Who is hiring?,"{""301:324"": 2}",CNN affiliates report on where job seekers are...
8,Iran criticizes who?,"{""63:97"": 2}","TEHRAN, Iran (CNN) -- Iran's parliament speake..."


In [249]:
# loops through the rows and prints the question along with the first answer given
start_truth = []
end_truth = []
for i, row in df.iterrows():
    try:
        answers = ast.literal_eval(row['validated_answers'])
        sorted_ans = sorted(answers.items(), key=operator.itemgetter(1), reverse=True)
        #print(sorted_ans)
        start, end = sorted_ans[0][0].split(':')
        start_truth.append(int(start))
        end_truth.append(int(end))
    except ValueError:
        start_truth.append(np.nan)
        end_truth.append(np.nan)
        pass
df['start_truth'] = start_truth
df['end_truth'] = end_truth
df = df.dropna()
print(len(df))
df.head()

80


,question,validated_answers,story_text,start_truth,end_truth
0,What was the amount of children murdered?,"{""none"": 1, ""294:297"": 2}","NEW DELHI, India (CNN) -- A high court in nort...",294.0,297.0
2,who did say South Africa did not issue a visa ...,"{""839:853"": 1, ""103:127"": 2}",Johannesburg (CNN) -- Miffed by a visa delay t...,103.0,127.0
4,What frightened the families?,"{""688:791"": 2, ""690:742"": 1}","BAGHDAD, Iraq (CNN) -- At least 6,000 Christi...",688.0,791.0
6,Who is hiring?,"{""301:324"": 2}",CNN affiliates report on where job seekers are...,301.0,324.0
8,Iran criticizes who?,"{""63:97"": 2}","TEHRAN, Iran (CNN) -- Iran's parliament speake...",63.0,97.0


In [291]:
final = df.drop(columns=['validated_answers'])
final.head()

,question,story_text,start_truth,end_truth
0,What was the amount of children murdered?,"NEW DELHI, India (CNN) -- A high court in nort...",294.0,297.0
2,who did say South Africa did not issue a visa ...,Johannesburg (CNN) -- Miffed by a visa delay t...,103.0,127.0
4,What frightened the families?,"BAGHDAD, Iraq (CNN) -- At least 6,000 Christi...",688.0,791.0
6,Who is hiring?,CNN affiliates report on where job seekers are...,301.0,324.0
8,Iran criticizes who?,"TEHRAN, Iran (CNN) -- Iran's parliament speake...",63.0,97.0


In [292]:
full_text = ''
m = 0
for col in ['question','story_text']:
    for text in final[col]:
        m = max(m, len(text))
        full_text += text.lower()
    print(m)
    m=0

# get the set of all characters
characters = tuple(set(full_text))
characters[:5]

68
7361


('a', 'h', 'm', '(', 'b')

In [293]:
# use enumeration to give the characters integer values
int2char = dict(enumerate(characters))

# create the look up dictionary from characters to the assigned integers
char2int = {char: index for index, char in int2char.items()}
print(char2int)


{'a': 0, 'h': 1, 'm': 2, '(': 3, 'b': 4, '\n': 5, 'y': 6, ' ': 7, '.': 8, 'g': 9, '4': 10, '2': 11, '6': 12, '-': 13, '$': 14, 'q': 15, 'v': 16, '5': 17, 'ñ': 18, 'c': 19, '*': 20, '3': 21, '8': 22, ':': 23, '[': 24, '_': 25, '7': 26, ',': 27, 'd': 28, '9': 29, 'â': 30, 'o': 31, 'j': 32, '"': 33, '!': 34, ']': 35, 'w': 36, 'u': 37, "'": 38, '1': 39, '0': 40, 't': 41, 's': 42, 'f': 43, 'é': 44, 'ã': 45, 'n': 46, 'e': 47, 'z': 48, 'ë': 49, '&': 50, 'l': 51, 'r': 52, 'p': 53, '•': 54, ')': 55, '/': 56, '?': 57, 'i': 58, ';': 59, 'k': 60, 'x': 61, '»': 62, '+': 63, '©': 64, 'ú': 65}


In [294]:
q_max = 68
story_max = 7361

In [295]:
padding_start = False

for i, row in final.iterrows():
    for col, N in zip(['question','story_text'], [q_max, story_max]):
        enc = np.zeros(N, dtype=np.int32)
        
        row[col] = np.array([char2int[char] for char in row[col].lower()])
        l = min(N, len(row[col]))
        if padding_start:
            enc[:l] = row[col][:l]
        else:
            enc[N-l:] = row[col][:l]
        final[col][i] = enc
        
final.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,question,story_text,start_truth,end_truth
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",294.0,297.0
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 36,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",103.0,127.0
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",688.0,791.0
6,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",301.0,324.0
8,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",63.0,97.0


In [296]:
def find_s(seq):
    for i, char in enumerate(seq):
        if char == 0:
            continue
        else: return i
        
find_s([0,0,0,0,0,0,1,2,4])

6

## Train/Test Split

In [297]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final[['question', 'story_text']], 
                                                    final[['start_truth','end_truth']], 
                                                    test_size=0.3, 
                                                    random_state=42)

In [298]:
print(X_train.index)
X_train.head()

Int64Index([ 85, 170, 118, 148,  31, 112, 152, 226,  13, 138, 146,  39, 187,
             54, 120,  22, 223,   6,  35, 109,  15, 183,  11,  94, 213, 159,
            126, 225,  26,  66, 116,  63, 132,  48, 119, 224, 154,  18,  80,
            220, 160, 174, 196, 105,  69,   2, 144,  42,   4,  47, 219,  40,
            161, 211,  25, 139],
           dtype='int64')


,question,story_text
85,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
170,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
118,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
148,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
31,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [496]:
min(y_train.end_truth)

16.0

In [299]:
y_train.head()

,start_truth,end_truth
85,2601.0,2606.0
170,1250.0,1308.0
118,64.0,98.0
148,1765.0,1777.0
31,1873.0,1907.0


In [388]:
class NewsQADataset(Dataset):
    def __init__(self, X, y, N=400, padding_start=False):
        self.start = y['start_truth'].values
        self.end = y['end_truth'].values
        self.question = X['question'].values
        self.story_text = X['story_text'].values
        
    def __len__(self):
        return len(self.question)
    
    def __getitem__(self, idx):
        #print(self.question)
        #qs = np.array([find_s(seq) for seq in self.question])
        ss = np.array([find_s(seq) for seq in self.story_text])
        return self.question[idx], self.story_text[idx], self.start[idx] + ss[idx], self.end[idx] + ss[idx]

In [389]:
train_ds = NewsQADataset(X_train, y_train)
test_ds = NewsQADataset(X_test, y_test)

In [390]:
train_ds[0]

(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 58, 46,  7,
        36,  1,  0, 41,  7,  6, 47,  0, 52,  7, 28, 58, 28,  7,  4,  2, 36,
         7, 47, 46, 41, 47, 52,  7, 41,  1, 47,  7, 42, 53, 31, 52, 41, 57],
       dtype=int32),
 array([ 0,  0,  0, ...,  0, 28,  8], dtype=int32),
 5303.0,
 5308.0)

In [391]:
batch_size = 8
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=batch_size)

In [392]:
q, s, start, end = next(iter(train_dl))
print(q.shape, s.shape, start.shape, end.shape)
q, s, start, end

torch.Size([8, 68]) torch.Size([8, 7361]) torch.Size([8]) torch.Size([8])


(tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0, 36,  1,  0, 41,  7, 43, 52, 58,  9,  1, 41, 47, 46, 47, 28,
           7, 41,  1, 47,  7, 43,  0,  2, 58, 51, 58, 47, 42, 57],
         [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0, 36,  1,  0, 41,  7, 36, 58, 51, 51,  7, 46, 47, 47, 28,
           7, 19, 52, 47,  0, 41, 58, 16, 47,  7, 28, 58, 53, 51, 31,  2,  0, 19,
           6,  7,  0, 46, 28,  7, 19, 31, 37, 52,  0,  9, 47, 57],
         [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0, 36,  1,  0, 41,  7,  1,  0, 53, 53,
          47, 46, 47, 28,  7, 41, 31,  7, 41,  1, 47,  7, 19, 58, 16, 58, 51, 58,
           0, 46,  7,  1, 47, 51, 58, 19, 31, 53, 41, 47, 52, 57],
         [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,

In [514]:
class GRUModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(GRUModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.dropout = nn.Dropout(0.5)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear2 = nn.Linear(200, 7361)
        self.linear_start = nn.Linear(7361, 7362)
        self.linear_end = nn.Linear(7361, 7362)
        
    def forward(self, x1, x2):
        
        x1 = self.embeddings(x1)
        x1 = self.dropout(x1)
        out_pack1, ht1= self.gru(x1)
        
        x2 = self.embeddings(x2)
        x2 = self.dropout(x2)
        out_pack2, ht2= self.gru(x2)
        
        
        x = torch.cat((ht1[-1], ht2[-1]),1)
        
        x = F.relu(self.linear2(x))
        start = self.linear_start(x)
        end = self.linear_end(x)
 
        return start, end

I've taken the functions from her last name classification model because that uses character-level embeddings as well. I'm using a naive version that just concats the question with the story and uses that as input into the RNN. I used that with regressing 2 outputs (start, end) and I got it to train but not sure how well it's actually doing. I think we correct way to do this is to treat it as a classification problem where each index of the story is a categroy and we use those predictions

In [456]:
def get_optimizer(model, lr = 0.1, wd = 0.0001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optim = torch.optim.Adam(parameters, lr=lr, weight_decay=wd)
    return optim

def vectorize(labels, size=7361):
    result = []
    for s in labels:
        #print(s)
        row = np.zeros(size)
        row[int(s)] = 1
        result.append(row)
        
    return result

vectorize([1, 10, 2])

[array([0., 1., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 1., ..., 0., 0., 0.])]

In [515]:
def train_epocs(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for q, s, start_, end_  in train_dl:
            #start_vec = vectorize(start_)
            #end_vec = vectorize(end_)

            start, end = model(q.long(), s.long())
            loss_start = F.cross_entropy(start, start_.long())

            loss_end = F.cross_entropy(end, end_.long())
            loss = loss_start + loss_end
            
        
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss = val_metrics(model, test_dl)
        #if i % 5 == 1:
        print("train loss %.3f val loss %.3f and val accuracy NA.3f" % (sum_loss/total, val_loss))
            
def val_metrics(model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for q, s, start_, end_ in valid_dl:
        start_vec = vectorize(start_)
        end_vec = vectorize(end_)

        start, end = model(q.long(), s.long())
        #print(start, end)
        #print(start.shape, end.shape)
        
        loss_start = F.cross_entropy(start, start_.long())
        #print(loss_start)
        
        loss_end = F.cross_entropy(end, end_.long())
        #print(loss_end)
        
        loss = loss_start + loss_end
        #print(loss)
        #
        start_i = start.argmax(1)
        end_i = end.argmax(1)
        
        
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    print(f'Predicted: {start_i}, Real: {start_}')
    #print(f'Real: {start_}, {end_}')
    return sum_loss/total

In [516]:
vocab_size = 66
hidden_size = 100
n_classes = 2
emb_size = 10
model = GRUModel(vocab_size, emb_size, hidden_size)

val_metrics(model, test_dl)

Predicted: tensor([2535, 6642, 2535, 2535, 2535, 2535, 2535, 2535]), Real: tensor([4778., 3379., 6393., 6404., 5393., 6702., 4002., 5283.],
       dtype=torch.float64)


17.806418736775715

In [ ]:
train_epocs(model)

Predicted: tensor([6458, 5358, 3171, 2923, 3171, 3171, 6458, 6458]), Real: tensor([4778., 3379., 6393., 6404., 5393., 6702., 4002., 5283.],
       dtype=torch.float64)
train loss 17.869 val loss 18.022 and val accuracy NA.3f
Predicted: tensor([2923, 5358, 5271, 2923, 3171, 3171, 1935, 2923]), Real: tensor([4778., 3379., 6393., 6404., 5393., 6702., 4002., 5283.],
       dtype=torch.float64)
train loss 11.474 val loss 31.311 and val accuracy NA.3f
Predicted: tensor([5777, 4953, 6487, 5167, 6487, 6487, 6487, 5493]), Real: tensor([4778., 3379., 6393., 6404., 5393., 6702., 4002., 5283.],
       dtype=torch.float64)
train loss 9.220 val loss 46.762 and val accuracy NA.3f
Predicted: tensor([5413, 5632, 5413, 5167, 5413, 5413, 5413, 5584]), Real: tensor([4778., 3379., 6393., 6404., 5393., 6702., 4002., 5283.],
       dtype=torch.float64)
train loss 8.577 val loss 39.984 and val accuracy NA.3f
Predicted: tensor([5119, 5632, 5413,  812, 5922, 5922, 5922, 4651]), Real: tensor([4778., 3379., 6393.